<b>Creating face data and storing in folder<b>

In [ ]:
import cv2


#create the model that detects human face
face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# after detecting face crop it
# this will extract_your cropped face
def face_extract(pic):
    gray = cv2.cvtColor(pic,cv2.COLOR_BGR2GRAY)
    
    faces = face_model.detectMultiScale(gray,1.2,5)   # (img , scaling , no of max faces)
    
    if faces is ():
        return None # if no face found--> return back
    
    for (x,y,w,h) in faces:
        cropped_face = gray[y:y+h,x:x+w]
        
    return cropped_face # if face found--> return cropped_face

# open your shutter
cap = cv2.VideoCapture(0)
count=0 # for counting no of images

while True:
    ret, pic = cap.read()
    
    if face_extract(pic) is not None:  # calling function
    
        count+=1
        pic = cv2.resize(face_extract(pic), (200,200))  # resizing your 480x640 --> 200x200

        file_name_path = "./img/" + str(count) + ".jpg"      #YOU HAVE TO CREATE img FOLDER N THEN GIVE ITS LOCATION
        cv2.imwrite(file_name_path, pic)  # save pic to folder

        cv2.putText(pic,str(count),(100,25),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('hi',pic) # show the live cropped_face

    else:
        print("Face not Found")  
        
    if cv2.waitKey(10)==13 or count==50:
        break
        
cv2.destroyAllWindows()
cap.release()


<b>Creating the model<b>

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join

fold_path="./img/"
list_of_files = [ f for f in listdir(fold_path) if isfile(join(fold_path,f))]

Training_dataset, Lables = [], []

for i, files in enumerate(list_of_files):
    img_path = fold_path + list_of_files[i]
    images = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    Training_dataset.append( np.asarray(images, dtype=np.uint8) )
    Lables.append(i)


rahul_model = cv2.face_LBPHFaceRecognizer.create()
rahul_model.train( np.asarray(Training_dataset), np.asarray(Lables))
print("Model created Successfully..!!")

<B>TESTING THE MODEL IF IT RECOGNISES ME OR NOT<B>

In [ ]:
import cv2
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 2)
    if faces is ():
        return img, [] , 0
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        
    return img, roi, (x,y-20)


cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()
    cv2.imwrite("pic.jpg", frame)
    
    image, face, dim = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        results = rahul_model.predict(face)
        
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            #display_string = str(confidence) + "%" 

        if confidence > 85:
            cv2.putText(image, "Hey Rahul: {}%".format(confidence), dim, cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
            print('YOU are Rahul')
    
        else:
            cv2.putText(image, "Plz go in light...", (220,80), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 80) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     



<b>IMPLEMENTING THE TASK..<b>

In [ ]:
import imghdr
import time
import smtplib  
from email.message import EmailMessage
import pywhatkit
import os
import subprocess as sp
import json

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 2)
    if faces is ():
        return img, [] , 0
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        
    return img, roi, (x,y-20)

    
def send_img_mail():
    From = "Face_Detector_Machine"
    Reciever_Email = "rcooxxxx@gmail.com"   #Type email to whom you want to mail

    msg = EmailMessage()                         
    msg['Subject'] = "Your face has detected..!" 
    msg['From'] = From
    msg['To'] = Reciever_Email 

    time.sleep(3)


    with open('pic.jpg', 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    msg.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)


    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login("hxxxxx@gmail.com", "mxxxxxx")
        server.send_message(msg)
        
    print("MAIL HAVE BEEN SENT SUCCESSFULLY...!")
    
def aws():
    print("Launching instance..")
    instance_id=sp.getoutput("aws ec2 run-instances --image-id  ami-0ad704c126371a548 --instance-type t2.micro --count 1 --subnet-id subnet-a80978e4 --security-group-ids sg-0ff45a6f66ccf70gb --key-name awskey")
    print("Instance launched..creating volume...")
    ins=json.loads(instance_id)
    volid=sp.getoutput("aws ec2  create-volume --size 5  --availability-zone  ap-south-1b  --snapshot-id snap-136ecgfa")
    VolId=json.loads(volid)
    time.sleep(60)
    vol_attach=sp.getoutput("aws ec2 attach-volume --volume-id  {}    --instance-id  {}   --device /dev/xvdh ".format(VolId['VolumeId'],ins['Instances'][0]['InstanceId']))
    print("Volume Attached Successfully..")
    print("DONE!")


    
    




# Open Webcam
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()
    cv2.imwrite("pic.jpg", frame)
    
    image, face, dim = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        results = rahul_model.predict(face)
        
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            #display_string = str(confidence) + "%" 
            
        if confidence > 85:
            cv2.putText(image, "Hey Rahul: {}%".format(confidence), dim, cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
            print('YOU are Rahul')
            print("Sending Mail..")
            send_img_mail()
            
            print("Preparing for whatsapp msg..")
            pywhatkit.sendwhatmsg("+9163xxxxxxx8", "Face_caputered" ,17,59,wait_time=10)
            #pywhatkit.sendwhatmsg("+MobileNumber", "Message" ,hr,min,wait_time(sec))
            
            break
    
        else:
            cv2.putText(image, "Plz go in light...", (220,80), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
            print("Somebody else face has been detected..")
            print("Launching instance in your AWS Cloud!")
            aws()
            break

    except:
        cv2.putText(image, "No Face Found", (220, 80) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     


